In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

/tmp/ipykernel_1789/2820714271.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
# Leeftijdsgroepen: download RIVM casus-data
@run
def cell():
  global rivm
  rivm = RIVM.csv('COVID-19_casus_landelijk')
  display(rivm.head())

downloading rivm/COVID-19_casus_landelijk-2022-04-29@13-15.csv


rivm COVID-19_casus_landelijk zipping rivm/COVID-19_casus_landelijk-2022-04-29@13-15.csv


loading rivm/COVID-19_casus_landelijk-2022-04-29@13-15.csv.gz


,Version,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Deceased,Week_of_death,Municipal_health_service
0,4,2022-04-29 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,NaN,GGD Amsterdam
1,4,2022-04-29 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,4,2022-04-29 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,NaN,GGD Hollands-Midden
3,4,2022-04-29 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,NaN,GGD Hollands-Noorden
4,4,2022-04-29 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,No,NaN,GGD Gelderland-Zuid


In [3]:
# Leeftijdsgroepen: download CBS bevolkingscijfers, uitgesplitst op de leeftijdscategorien zoals bij RIVM
@run
def cell():
  global bevolking
  # probeer de laatste leeftijdsgroepen op te halen bij het CBS, ververs de fallback-file 
  try:
    bevolking = CBS.bevolking(leeftijdsgroepen=True)
    bevolking.to_csv("cbs/leeftijdsgroepen_cbs.csv")
  # als het niet is gelukt de CBS file op te halen, gebruik de fallback-file en pas de index klom aan
  except Exception as e:
    print(e)
    bevolking = pd.read_csv("cbs/leeftijdsgroepen_cbs.csv")
    bevolking.set_index('Range', inplace=True)
  display(bevolking.head())

/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


,BevolkingOpDeEersteVanDeMaand,per 100k
Range,,
0-9,1761622,0.056766
10-19,1976813,0.050586
20-29,2278330,0.043892
30-39,2239565,0.044652
40-49,2127237,0.047009


In [4]:
# Leeftijdsgroepen: bereken per cohort besmettingen / opnamen / sterfte (incl. kleurcode), in aantal en per 100k. Vervang <50 en Unknown door Onbekend
@run
def cell():
  #  kolom is in version 2 per 18-1-22 toegevoegd, verwijderen:
  rivm.drop('Version', inplace=True, axis=1)
  display(rivm.head())
  rivm['Cohort'] = rivm['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
  # aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
  assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in rivm['Cohort'].unique() if cohort[0].isdigit()]
  assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
  rivm['Gemiddelde leeftijd'] = rivm['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

  # verwijder tijd
  rivm['Date_file_date'] = pd.to_datetime(rivm['Date_file'].replace(r' .*', '', regex=True))

  rivm['Date_statistics_date'] = pd.to_datetime(rivm['Date_statistics'])

  # weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
  rivm['Weken terug'] = np.floor((rivm['Date_file_date'] - rivm['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(int)

  # voeg key, gem leeftijd, kleurnummer en totaal toe
  Date_file = rivm['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
  cohorten = list(bevolking.index) + ['Onbekend']

  def summarize(df, category, prefix):
    # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
    # en we het 'rivm' frame later nog clean nodig hebben
    df = df.copy(deep=True)

    df = (df
          .groupby(['Weken terug', 'Cohort'])['count']
          .sum()
          .unstack(fill_value=np.nan)
          .reset_index()
          .rename_axis(None, axis=1)
        ).merge(df
          # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
          # niveau aggregeren voor 'df' overschreven word
          .groupby(['Weken terug'])['Gemiddelde leeftijd']
          .mean()
          .to_frame(), on='Weken terug'
        )

    # altijd 52 rijen
    df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

    # toevoegen missende cohorten
    for col in cohorten:
      if not col in df:
        df[col] = np.nan

    # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen
    df['Totaal'] = df[cohorten].sum(axis=1)

    # voeg periode en datum toe
    # periode afgeleid van weken-terug (= de index voor deze dataframe)
    df['Datum'] = pd.to_datetime(Date_file)
    df['Periode'] = (df
      .index.to_series()
      .apply(
        lambda x: (
          (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
          + '-'
          + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
        )
      )
    )

    # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
    df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
    df['Type'] = category

    # voeg de kleur kolommen toe
    for col in cohorten:
      df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

    # herschikken van de kolommen
    colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
    return df[colorder + [col for col in df if col not in colorder]]

  factor = bevolking.to_dict()['per 100k']

  global tabel
  tabel = pd.concat(
    # flatten the result list zodat pd.concat ze onder elkaar kan plakken
    functools.reduce(lambda a, b: a + b, [
      [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
      for df, label, prefix in [
        (rivm, 'Positief getest', 'p'), # volledige count per cohort
        (rivm[rivm.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
      ]
    ])
  )

  # bij /100k is het Totaal-veld geen optelsom maar wordt berekend obv de gehele bevolking
  totale_bevolking_per_cohort = bevolking.to_dict()['BevolkingOpDeEersteVanDeMaand']
  totale_bevolking = sum(totale_bevolking_per_cohort.values())
  keys = [ key for key in tabel["Key"]]
  totals = [ x for x in tabel["Totaal"]]
  for k in range(0, len(keys)):
    key = keys[k]
    if '100k' in key:
      abskey = key[0] + key[5:]
      kk = keys.index(abskey)
      if isinstance(totals[k], float):
        correctedtotal = totals[kk] * (100_000 / totale_bevolking)
        # print([k, totals[k], totals[kk], correctedtotal])
        totals[k] = correctedtotal
  tabel["Totaal"] = totals

  display(tabel.head())

,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Deceased,Week_of_death,Municipal_health_service
0,2022-04-29 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,NaN,GGD Amsterdam
1,2022-04-29 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2022-04-29 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,NaN,GGD Hollands-Midden
3,2022-04-29 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,NaN,GGD Hollands-Noorden
4,2022-04-29 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,No,NaN,GGD Gelderland-Zuid


,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,...,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2022-04-29,23/04-29/04,45.836111,7200.0,Positief getest,163.0,353.0,1225.0,...,249,866,969,702,1000,568,351,193,74,0
1,p001,1,2022-04-29,16/04-22/04,45.847218,16985.0,Positief getest,285.0,926.0,2805.0,...,285,866,1000,787,982,630,336,199,66,0
2,p002,2,2022-04-29,09/04-15/04,45.572413,35395.0,Positief getest,821.0,1981.0,5495.0,...,290,806,985,785,1000,662,313,171,59,0
3,p003,3,2022-04-29,02/04-08/04,45.624083,81080.0,Positief getest,2205.0,4802.0,11658.0,...,314,764,988,816,1000,732,358,146,48,0
4,p004,4,2022-04-29,26/03-01/04,46.923334,161353.0,Positief getest,5160.0,10014.0,20548.0,...,342,702,925,798,1000,850,506,170,44,0


In [5]:
if knack:
  await knack.publish(tabel.fillna(0).assign(Datum=tabel.Datum.dt.strftime('%Y-%m-%d')), 'Leeftijdsgroep', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


updating knack


...

updating: [{'Key': 1, 'Updating Leeftijdsgroep': 'false'}]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:09<00:00,  9.82s/it]

100%|██████████| 1/1 [00:09<00:00,  9.82s/it]


rate limit: 7 
LaatsteUpdate API calls: create: 0, read: 0, update: 1, delete: 0, backoff: 0
